In [8]:
pip install scikit-tda

Note: you may need to restart the kernel to use updated packages.


You are using pip version 19.2.3, however version 19.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from numpy import genfromtxt
from ripser import ripser,lower_star_img
from persim import plot_diagrams

In [22]:
letters = genfromtxt('1) letters.csv', delimiter =',')


#the main block: run 5 scans for each of the 26 letters, store results in a dictionary.
i=0
i_to_result = dict()
while i<=25:
    letter_one_line = letters[i,1:] #!!!change it! drop the first number of [12,0,0,0,1,0,0......]
#initialize matrix of size 10x10 with all values 100
    letter= np.full((10,10),100)
   #test 1 probing upper left
    for k in range(1,100):
        if letter_one_line[k]==1.0:
            row=int((k-1)/10)
            column= (k-1)%10
            letter[row,column]= max((k-1)%10, int((k-1)/10))
    PUscan = lower_star_img(letter)
    #test 2 left to right
    for j in range(1,100):
        if letter_one_line[j]==1.0:
            row=int((j-1)/10)
            column= (j-1)%10
            letter[row,column]= j%10
    LRscan = lower_star_img(letter)
    #test 3 right to left
    for k in range(1,100):
        if letter_one_line[k]==1.0:
            row=int((k-1)/10)
            column= (k-1)%10
            letter[row,column]= 10 - (k-1)%10
    RLscan = lower_star_img(letter)
    #test 4 probing from bottom right
    for k in range(1,100):
        if letter_one_line[k]==1.0:
            row=int((k-1)/10)
            column= (k-1)%10
            letter[row,column]= max(9-(k-1)%10,9-int((k-1)/10))  
    PLBscan = lower_star_img(letter)
    #test 5 probing from upper right to lower left 
    for k in range(1,100):       
        if letter_one_line[k]==1.0:
            row=int((k-1)/10)
            column= (k-1)%10
            letter[row,column]= max(9-(k-1)%10,int((k-1)/10))  
    PURscan = lower_star_img(letter) 
    i_to_result[i] = (PUscan, LRscan, RLscan, PLBscan, PURscan)
    i+=1

#change all infinity in life-death pair to 100.    
l=0
while l <=25:
    for j in range(5):
        len_j = len(i_to_result[l][j])
        for k in range(len_j):
            if str(i_to_result[l][j][k][1])=='inf':
                i_to_result[l][j][k][1]=100
    l+=1

#calculate feature vector for each letter, store them in a matrix. 
#every feature vector has 5 components, each component stands for the sum of "lifespan" of the life-death pairs in one scan.
l=0
vec_mtx = np.zeros((26,10))
while l < 26:
    for j in range(5):
        len_j = len(i_to_result[l][j])
        sum_j = 0
        for k in range(len_j):
            sum_j = sum_j+ (i_to_result[l][j][k][1]-i_to_result[l][j][k][0])
            vec_mtx[l][j] = sum_j
    l+=1
print('feature matrix:\n',vec_mtx,'\n')

#calculate the pairwise distance between each two letter, store them in a list.
all_dis = []
for q in range(26):
    for p in range(q):
            dis = np.linalg.norm(vec_mtx[q]-vec_mtx[p])
            all_dis.append((dis,p,q))

#sort all the distance from low to high, find the minimum distance, also find which letter pair has the minimum distance.             
def getkey(item):
    return item[0]

sorted_dis = sorted(all_dis, key=getkey)
print('how many items in this sorted distance list:',len(sorted_dis),'\n','sorted distance:',sorted_dis,'\n')

print('min distance is between',sorted_dis[0][1],'th letter and',sorted_dis[0][2],'th letter. min distance is',sorted_dis[0][0],'\n')




the ith: 0 [[ 2. inf]]
the ith: 1 [[ 1. inf]]
the ith: 2 [[ 2. inf]]
the ith: 3 [[ 1. inf]]
the ith: 4 [[ 1. inf]]
the ith: 5 [[ 1. inf]]
the ith: 6 [[ 1. inf]]
the ith: 7 [[ 1. inf]]
the ith: 8 [[ 2. inf]]
the ith: 9 [[ 2. inf]]
the ith: 10 [[ 1. inf]]
the ith: 11 [[ 1. inf]]
the ith: 12 [[ 1. inf]]
the ith: 13 [[ 1. inf]]
the ith: 14 [[ 1. inf]]
the ith: 15 [[ 4.  5.]
 [ 1. inf]]
the ith: 16 [[ 6.  7.]
 [ 1. inf]]
the ith: 17 [[ 1. inf]]
the ith: 18 [[ 1. inf]]
the ith: 19 [[ 1. inf]]
the ith: 20 [[ 6.  8.]
 [ 1. inf]]
the ith: 21 [[ 5.  6.]
 [ 1. inf]]
the ith: 22 [[ 3.  5.]
 [ 1. inf]]
the ith: 23 [[ 1. inf]]
the ith: 24 [[ 4.  5.]
 [ 1. inf]]
the ith: 25 [[ 4.  5.]
 [ 1. inf]]
feature matrix:
 [[ 98.  99.  96.  97.  96.]
 [ 99.  98.  97.  97.  97.]
 [ 98.  99. 101.  98.  98.]
 [ 99.  98.  97.  97.  97.]
 [ 99.  98. 106. 100. 101.]
 [ 99.  98.  99.  95. 100.]
 [ 99.  99.  99.  96.  99.]
 [ 99.  98.  95.  96.  96.]
 [ 98.  98.  95.  95.  95.]
 [ 98.  99.  94.  95.  95.]
 [ 99.  98. 

In [11]:
#convert new test input list of 01010101... into feature vector.
def get_feature_vec(test): # 'test' stands for the 01010101... input.
    #run all the scans, get the life-death pairs.
    letter= np.full((10,10),100)
   #test 1 probing upper left
    for k in range(1,100):
        if test[k]==1.0:
            row=int((k-1)/10)
            column= (k-1)%10
            letter[row,column]= max((k-1)%10, int((k-1)/10))
    PUscan = lower_star_img(letter)
    #test 2 left to right
    for j in range(1,100):
        if test[j]==1.0:
            row=int((j-1)/10)
            column= (j-1)%10
            letter[row,column]= j%10
    LRscan = lower_star_img(letter)
    #test 3 right to left
    for k in range(1,100):
        if test[k]==1.0:
            row=int((k-1)/10)
            column= (k-1)%10
            letter[row,column]= 10 - (k-1)%10
    RLscan = lower_star_img(letter)
    #test 4 probing from bottom right
    for k in range(1,100):
        if test[k]==1.0:
            row=int((k-1)/10)
            column= (k-1)%10
            letter[row,column]= max(9-(k-1)%10,9-int((k-1)/10))  
    PLBscan = lower_star_img(letter)
    #test 5 probing from upper right to lower left 
    for k in range(1,100):       
        if test[k]==1.0:
            row=int((k-1)/10)
            column= (k-1)%10
            letter[row,column]= max(9-(k-1)%10,int((k-1)/10))  
    PURscan = lower_star_img(letter) 
    test_result = (PUscan, LRscan, RLscan, PLBscan, PURscan)
        
    #change all infini to 100
    for j in range(5):
        len_j = len(test_result[j])
        for k in range(len_j):
            if str(test_result[j][k][1])=='inf':
                test_result[j][k][1]=100
    
    #get the feature vector
    test_feature = np.zeros(5)
    for j in range(5):
        len_j = len(test_result[j])
        sum_j = 0
        for k in range(len_j):
            sum_j = sum_j+ (test_result[j][k][1]-test_result[j][k][0])
            test_feature[j] = sum_j
            
    return test_feature

#make up a test list. I wrote the letter S (the 18th letter) in 10X10 grid, and recored the coordinates in one_list.
New_S = [0]*100
one_list = [14,15,16,17,23,27,33,43,44,45,46,56,57,67,73,74,76,77,84,85,86]

for i in one_list:
    New_S[i]=1
    
feature_vect = get_feature_vec(New_S)

print('feature vect of input list is:',feature_vect)
print('standard feature vect of S is:', vec_mtx[18])

#compare the newly obtained test feature vector, to each of the feature vector stored in matrix.
def find_new_dis(feature):
    test_dis = []
    for i in range(len(vec_mtx)):
    # compare feature_vect with each of the vect in vec_mtx, see which distance=norm(feature_vect - vect) is smalletst.
        d = np.linalg.norm(feature - vec_mtx[i])
        test_dis.append((d,i))
    return test_dis

new_dis = find_new_dis(feature_vect)
    
#then find the which vector has the smallest distance with input test vector, choose it as our best fitted letter.
def sort_dis(dis_list):
    sorted_list = sorted(dis_list,key=getkey)
    return sorted_list

sorted_result = sort_dis(new_dis)

print('the best fitted letter is:',sorted_result[0][1],'th letter','\n','the closest distance is:',sorted_result[0][0],'\n',
     'other option:',sorted_result[1],sorted_result[2],sorted_result[3],'\n')

print('full resulting distance:','\n',sorted_result)

feature vect of input list is: [ 99. 101. 100.  98. 101.]
standard feature vect of S is: [ 99. 103. 101.  97. 100.]
the best fitted letter is: 18 th letter 
 the closest distance is: 2.6457513110645907 
 other option: (3.605551275463989, 6) (3.872983346207417, 2) (4.47213595499958, 5) 

full resulting distance: 
 [(2.6457513110645907, 18), (3.605551275463989, 6), (3.872983346207417, 2), (4.47213595499958, 5), (4.69041575982343, 22), (4.69041575982343, 25), (5.291502622129181, 16), (5.477225575051661, 10), (5.916079783099616, 1), (5.916079783099616, 3), (6.082762530298219, 12), (6.082762530298219, 14), (6.244997998398398, 23), (6.4031242374328485, 21), (6.6332495807108, 17), (6.782329983125268, 20), (6.855654600401044, 0), (7.0, 4), (7.14142842854285, 15), (7.211102550927978, 19), (7.615773105863909, 13), (7.937253933193772, 7), (8.306623862918075, 24), (8.94427190999916, 8), (9.273618495495704, 9), (10.908712114635714, 11)]


In [16]:
#THe noise generating block

letters = genfromtxt('1) letters.csv', delimiter =',')
letter_G = letters[6,1:]

G_feature = get_feature_vec(letter_G)
print('G feature:',G_feature,'\n',vec_mtx[6])

G_dis = find_new_dis(G_feature)
sort_G = sort_dis(G_dis)

# add random noise point to original letter G, see the result.
random_ones = np.random.randint(0,100,4)

add_one = np.zeros(100)
for i in random_ones:
    add_one[i]=1

Noise_G = letter_G + add_one

NG_feature = get_feature_vec(Noise_G)
print('Noise G:',NG_feature,'\n',vec_mtx[6])

NG_dis = find_new_dis(NG_feature)
sort_NG = sort_dis(NG_dis)
print('the new distance list:',sort_NG)

#conclusion: this method is not rebust against noise points. at. all.

G feature: [99. 99. 99. 96. 99.] 
 [99. 99. 99. 96. 99.]
Noise G: [281. 287. 289. 288. 281.] 
 [99. 99. 99. 96. 99.]
the new distance list: [(412.4051406081161, 4), (414.2052631244562, 18), (414.6263378030875, 25), (416.85249189611426, 2), (417.388308413161, 22), (417.79899473311326, 6), (418.26905216618644, 16), (418.2762245215475, 5), (418.68126301519635, 12), (419.1217007027911, 10), (419.1741881366266, 20), (419.5688262967114, 23), (419.57835978515385, 1), (419.57835978515385, 3), (419.58789305698514, 14), (419.61172528898663, 21), (420.01785676325716, 17), (420.46165104560964, 0), (420.51872728809593, 15), (420.9429890139519, 13), (420.9619935338581, 19), (421.3905551860412, 7), (421.41665842726246, 24), (422.71858251087093, 8), (422.732776112759, 9), (423.16190754839926, 11)]


In [21]:
# if we're tired of counting letters by fingers, use this number-letter converter.
i_list = list(range(26))
letter_list = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
i_letter_pair = list(zip(i_list,letter_list))


i_to_letter = {}
for i in range(26):
    i_to_letter[i]=letter_list[i]

letter_to_i={}
for i,letter in i_letter_pair:
    letter_to_i[letter]=i

letter_to_i['m']

12

In [21]:
i_to_letter[3]

'd'

In [28]:
ones_M1 = [14,15,24,25,29,34,35,38,39,43,44,45,48,50,53,55,56,58,60,63,65,67,70,72,76,77,80,82,86,90]
letter_M1 = np.zeros(100)
for i in ones_M1:
    letter_M1[i] = 1
    
M1_feature = get_feature_vec(letter_M1)
print('M1 feature vect:',M1_feature,'original M feature vect:',vec_mtx[12],'\n')

M1_dis = find_new_dis(M1_feature)
M1_sort = sort_dis(M1_dis)

print('sorted distance from M1:',M1_sort)


M1 feature vect: [ 97. 107.  99. 102.  99.] original M feature vect: [99. 99. 96. 99. 97.] 

sorted distance from M1: [(6.244997998398398, 25), (7.0710678118654755, 18), (9.273618495495704, 2), (9.486832980505138, 12), (9.643650760992955, 16), (10.198039027185569, 6), (10.295630140987, 14), (10.295630140987, 23), (10.392304845413264, 0), (10.44030650891055, 22), (10.723805294763608, 10), (10.862780491200215, 1), (10.862780491200215, 3), (11.045361017187261, 21), (11.090536506409418, 17), (11.357816691600547, 13), (11.61895003862225, 5), (11.61895003862225, 20), (11.916375287812984, 4), (12.041594578792296, 19), (12.083045973594572, 7), (12.328828005937952, 15), (12.449899597988733, 9), (12.767145334803704, 8), (12.806248474865697, 24), (13.638181696985855, 11)]
